<a href="https://colab.research.google.com/github/kytk/AI-MAILs/blob/main/python_5_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 医療従事者のためのPython: 深層学習を使った手書き数字の分類

根本清貴 (筑波大学医学医療系精神医学)

Ver. 20240730

## 本セクションの目標
- MNISTデータセットを用いて手書き数字の分類を行うことで、どのように深層学習を実装するかを学ぶ

## 目次
1. MNISTデータセット
2. Tensorflow/Keras を用いた深層学習の実装 (全結合層)
3. 学習の可視化
4. ハイパーパラメータ
5. 畳み込みニューラルネットワーク

## 1. MNISTデータセット
- NIST (National Institute of Standards and Technology) が保有していたデータセットを再構成したデータベース
- 60,000枚の訓練用画像と10,000枚の評価用画像が含まれている

| <img src="https://www.nemotos.net/nb/img/MnistExamples.png" width="500"> |
| --: |
| Wikipediaより引用 |

## 2. Tensorflow/Keras を用いた深層学習の実装 (全結合層)
- 深層学習を実装する手順は以下となる

| <img src="https://www.nemotos.net/nb/img/dl_flow.png" width="500"> |
| --: |
| 動かしながら学ぶPyTorchプログラミング入門より引用 |

- この流れに従っていく

- まずは、基本である「全結合層」のみを用いた識別を行い、その後、「畳みこみニューラルネットワーク」を用いた識別を行う

### 2-1. 必要なパッケージのインポート
- 今回必要なパッケージは以下
    - numpy: 行列計算
    - matplotlib: グラフの描画
    - tensorflow: 深層学習のフレームワーク
    - keras: tensorflowのフロントエンド（簡単なコマンドでTensorflowに仕事をさせるためのプログラム）

In [ ]:
# 必要なパッケージ、モジュールのインポート

# NumPy
import numpy as np

# Matplotlib
import matplotlib.pyplot as plt

# keras
from tensorflow import keras

# layers と Sequential (層　(layers) を定義し、層を順番 (sequential) につなげるために使用)
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

# ラベルを one-hotベクトルに変換する関数 to_categorical()
from tensorflow.keras.utils import to_categorical

### 2-2. データの前処理
- MNISTの画像データはひとつひとつのピクセルの値が0-255の値をとる
- これを0-1の値をとるように変換する

#### 2-2-1. データの読み込みと確認

In [ ]:
# tensorflow パッケージの中に mnist データセットが既に入っているため、それを利用する
from tensorflow.keras.datasets import mnist

In [ ]:
# mnist.load_data() でデータセットを読み込むことができる
# どのようなデータか確認してみる
# 変数 dataset を作成して代入する

dataset = mnist.load_data()

In [ ]:
# dataset のデータ型を確認する
# type(dataset) で dataset のデータ型が確認できる
# データ型はタプル
type(dataset)

In [ ]:
# dataset を実際に確認してみる
# よくみると

# ( (array画像, array数字), (array画像, array数字) )

# となっている
# (array画像, array数字) がかたまりとして1つで、2つの要素がある

dataset

In [ ]:
#タプルの場合、len で中に入る要素数を確認できる
# 今は2
# (array画像, array数字) のかたまりが 2つあるという意味
len(dataset)

In [ ]:
# 要素は 2 とわかったので、data0 と data1 にそれぞれ代入してみる
# タプルもインデックスを使って要素を取り出せる
# Python では、以下のように一行で2つの変数に代入できる
# (data0, data1 = dataset でも実は大丈夫)
data0, data1 = dataset[0], dataset[1]

In [ ]:
# data0, data1 についてもそれぞれ、type と len を使ってみる
print(f'data0: type {type(data0)} length {len(data0)}')
print(f'data1: type {type(data1)} length {len(data1)}')


In [ ]:
# data0 も data1 もタプルでさらに2つの要素があることがわかる
# それぞれさらに data00, data01, data10, data11 に代入してみる
data00, data01 = data0
data10, data11 = data1

print(f'data00: type {type(data00)} length {len(data00)}')
print(f'data01: type {type(data01)} length {len(data01)}')
print(f'data10: type {type(data10)} length {len(data10)}')
print(f'data11: type {type(data11)} length {len(data11)}')


- 今の作業でわかったこと
- data は 2つのタプルで構成
    - 各々のタプルはそれぞれ2つのNumpy配列で構成

- mnist.load_data() のデータ構成
    - 次のリンクに説明あり https://keras.io/ja/datasets/#mnist
    - data00: 60000枚の手書き画像をNumpy配列に変換したもの → 学習のための画像
    - data01: 60000枚の手書き画像の正解データ（数字）→ 学習のためのラベル
    - data10: 10000枚の手書き画像をNumpy配列に変換したもの → テストのための画像
    - data11: 10000枚の手書き画像の正解データ（数字） → テストのためのラベル

- 情報は改ざんされたくないので、タプルに入れる(タプルに入った内容は変更できない)

- これからデータ解析のために、改めて変数名をつけていく

- data00 は train_images, data01 は train_labels という名前にする
    - (train_images, train_labels) でひとつのタプルができる

- data10 は test_images, data11 は test_labels という名前にする
    - (test_images, test_labels) でもうひとつのタプルができる

- これらをまとめて、data というタプルができる
- **data = ((train_images, train_labels), (test_images, test_labels))**

- 結論: もらったデータについて知りたかったら、type 関数 と len 関数を使うことからはじめるとよいでしょう

In [ ]:
# mnist.load_data() を訓練データとテストデータにわけて格納する
# 様々な参考書はここからいきなり始まる(上はそれの説明だった)

# mnistは、訓練データとテストデータがそれぞれタプルにわかれて入っている
# 訓練データの画像を train_images, 正解ラベルを train_labels に格納する
# テストデータも同じ
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [ ]:
# train_images について確認する
# まず、型から確認
# numpy.ndarray型
type(train_images)

In [ ]:
# shape
# 60000枚の画像、1枚の画像は 28 x 28 で構成
train_images.shape

In [ ]:
# plt.imshow() を使って実際の画像を確認
# for を使って、最初の3人分のデータを見る
for i in range(3):
    plt.figure(figsize=(0.5,0.5)) #グラフのサイズを決定 0.5,0.5 は 0.5インチ x 0.5インチ
    plt.imshow(train_images[i], cmap='gray')
    plt.show()


In [ ]:
# train_labels の内容を確認
# スライシングで train_labels の最初の3つのラベルを取り出す
# 画像とラベルが一致していることを確認
train_labels[0:3] # train_labels の0番目以上、3番目未満を抽出

In [ ]:
# train_labels の shape を確認
# 60000 のデータがある1次元のデータ
train_labels.shape

In [ ]:
# 同様にテストデータも確認
# shape
# 10,000枚の画像、1枚の画像は 28 x 28 で構成
test_images.shape

In [ ]:
# plt.imshow() を使って実際の画像を確認
# for を使って、最初の3人分のデータを見る
for i in range(3):
    plt.figure(figsize=(0.5,0.5))
    plt.imshow(test_images[i], cmap='gray')
    plt.show()


In [ ]:
# test_labels の内容を確認
# スライシングで test_labels の最初の3つのラベルを取り出す
test_labels[0:3]

#### 2-2-2. 正解ラベルの one-hotベクトル化
- 正解ラベルを one-hotベクトルに変換する
- `to_categorical()` 関数を使うことで変換できる

In [ ]:
# train_labels を one-hotベクトルに変換
train_labels = to_categorical(train_labels)

# test_labels も同様に one-hotベクトルに変換
test_labels = to_categorical(test_labels)

In [ ]:
# 新しい train_labels の shape を確認
# 60000行10列の行列になっている
train_labels.shape

In [ ]:
# train_labels の 最初の3行を見てみる
# 正解 5, 0, 4 に相当するところが 1 になっていることに着目
train_labels[0:3]

In [ ]:
# test_labels の 最初の3行も確認する
# 正解 7, 2, 1 に相当するところが 1 になっていることに着目
test_labels[0:3]

#### 2-2-3. データの正規化
- 深層学習に限らず、データ解析においてデータの範囲をある決まった範囲に変換することを正規化という
- 正規化を行うことで、異なる変数がモデルに与える影響を均等にできる
    - 例: 年齢 (20-80) と 身長 (140-200)
- 今、画像は 0-255 の整数をとるので、これを 0-1 になるように変換する

In [ ]:
# dtype 属性で numpy配列内の数字のデータ型がわかる
# uint8 は unsigned integer 8bit 符号なし 8bit 整数 (0-255)
train_images.dtype

In [ ]:
# train_images の最小値と最大値
# min() メソッドとmax() メソッドを使えばよい
print(f'min: {train_images.min()}')
print(f'max: {train_images.max()}')

In [ ]:
# 0-255で構成されるので、255で割れば、値は 0-1 の間となる
# 255.0 と小数点をつけて割ることで、Pythonは出力を必ず float型としてくれる
# 計算結果を同じ変数名にいれることで、変数を増やすことなく、正規化されたデータに変数の内容が置き換わる
train_images = train_images / 255.0
test_images = test_images / 255.0

In [ ]:
# train_images の値が本当に0-1になったか確認
# min() と max() を使えばよい
# 最小値
print(f'min: {train_images.min()}')
print(f'max: {train_images.max()}')

In [ ]:
# dtypeも確認する
# 今回は float64 倍精度浮動小数点数
train_images.dtype

### 2-3. 訓練データとテストデータの作成
- MNISTデータセットは手書き数字6万枚の訓練データセットと手書き数字1万枚のテストデータセットから構成されている
- 「**訓練データ**」「**検証データ**」「**テストデータ**」の3つを準備する
    - 訓練データ: ニューラルネットワークのパラメータを決めるためのデータ
    - 検証データ: 訓練データで得られたパラメータがどの程度の精度があるかを検証するためのデータ
    - テストデータ: ニューラルネットワークの汎用性を評価するためのデータ
        - テストデータは、必ず訓練で使ったものと別のセットを使わないといけない
        - このため、訓練データの一部を検証データにする
- Tensorflow には、`model.fit()` メソッドに、`validation_split` という引数が準備されており、ここで訓練データの何割を検証データとして使用するかを設定できる
    - 今回は訓練データの2割を検証データとして使用することとする (`validation_split=0.2`)

### 2-4. ニューラルネットワークの定義

- ここで、自分がイメージするニューラルネットワークモデルを定義する
- 下図の赤線の部分, すなわち **順伝播 forwad propagation** のモデルを構築

<img src="https://www.nemotos.net/nb/img/dl_overview_4.png" width="500">

- 今は以下のように定義する
    - 層と層の結合は全結合とする
    - 第1層は画像が 28 x 28 で構成されているので、そのピクセル数(784)がユニット数 (数字のピクセルを1列に並べたイメージ)
    - 第2層のユニット数は 128 とする
    - 第2層の活性化関数は **ReLU**関数 とする
    - 過学習を防ぐため、全結合層の2割は drop とする (Dropout=0.2)
    - このモデルとしては、最終の出力は 0-9 の10のクラスを分類したい
    - そのため、第3層は 出力層に渡す準備として、ユニット数は 10 とする
    - 第3層の出力はそのまま出力層の入力とする (Tensorflow ではそのように構築することが勧められている)
    - 出力層の活性化関数は多クラス分類に適した **Softmax**関数 とする

- 活性化関数の特徴

| 関数名 | 特徴 |
| :-- | :-- |
| ReLU | 隠れ層に使うことで、非線形問題を解くことができるようになる <br> Sigmoid関数は0-1の値しかとらないので層が厚くなるほど誤差が小さくなっていき、<br>入力層まで誤差が伝搬する前に誤差が消失するという勾配消失問題が発生する |
| Sigmoid | 0-1の間の確率で表現可能なため2クラス分類の出力層に用いる |
| Softmax | 各クラスの確率の総和が1となるように正規化された関数であるため多クラス分類の出力層に用いる |

<img src="https://www.nemotos.net/nb/img/nn_model.png" width="500">

- これらはTensorflow/Kerasでは以下のように定義できる
    - プリセットで準備されている **Sequential**モデル を選択する(Sequential: 連続する)
    - 入力画像は **layers.Flatten** を使うことでベクトルにできる
    - 全結合層は **layers.Dense** で規定できる

In [ ]:
# モデルを定義
# Sequentialモデルを使用すると、model.addとすればどんどん付け加えられる
# ここはあくまでもモデルを作っているだけなので、データはまだ入力していない。（計算式を作るイメージ。データの代入はこれから）

model = Sequential()                                # Sequential() から model というオブジェクトを生成
model.add(layers.Flatten(input_shape=(28, 28)))     # 入力画像の 配列の大きさ(dimension) を指定 今の場合は(28,28)
model.add(layers.Dense(128, activation='relu'))     # 第2層のユニット数を 128 にし、 活性化関数は ReLU とする
model.add(layers.Dropout(0.2))                      # 過学習防止のため、結合層の20%を dropout
model.add(layers.Dense(10))                         # 第3層のユニット数を 10 にする ここは特に活性化関数はなし
model.add(layers.Softmax())                         # 出力層でSoftmax関数 で処理して結果を出力する


In [ ]:
# どのようなモデルになったかを model.summary() で知ることができる
model.summary()
# パラメータ数
#   Flatten: 入力層なのでなし
#   Dense 100480 ← 入力層 784 * 第2層 128 + 第2層のそれぞれのユニットに対する定数項 128
#   Dense 1290 ← 第2層 128 * 第3層 10 + 第3層のそれぞれのユニットに対する定数項 10
# 合計 101,770 ものパラメータをこれから学習させることになる
# パラメータが多いため、パラメータ推定のために必要なデータ数が膨大となる

In [ ]:
# 訓練データを今作ったモデルに代入して予測値を計算 (順伝播; forward propagation)
predictions = model(train_images)
# predictions の データ型を確認
print(type(predictions))

In [ ]:
# tensorflow.python.framework.ops.EagerTensor はそのままでは扱いにくいのでNumpy配列型に変換する
# predictions の後に .numpy() をつけることで、NumPy配列に変換できる
predictions = predictions.numpy()
# 型を確認しておく
print(type(predictions))

In [ ]:
# predictions を表示
# この時点では、モデルを決めただけで、重みはランダムに割り当てられている
# そのため、各クラスの確率はおおよそ 1/10 あたりになるはず
# ひとつの数字の画像が1行、列が 0 - 9 の数字である確率
# 一切学習はしていないことに注意！train_labelsはまだ使われていない

# np.round() は np.round(a, 2) で a を小数点2桁で丸めている

print(np.round(predictions, 2))

### 2-5. 損失関数と最適化関数の定義
- 次に損失関数と最適化関数(オプティマイザ)を決定する
- その後、model.compile() で損失関数と最適化関数をモデルに組み込む
- 下図の赤線の部分、すなわち **逆伝播 back propagation** のモデルを構築

<img src="https://www.nemotos.net/nb/img/dl_overview_5.png" width="500">

- よく用いられる損失関数

| 目的 | 関数名 | Function Name | 損失関数名<br>(Tensorflow) | 損失関数名(PyTorch) |
| :-- | :-- | :-- | :-- | :-- |
| 回帰 | 平均二乗誤差 | Mean Squared Error | mean_squared_error | nn.MSELoss |
| 2クラス分類 | バイナリ交差エントロピー | Binary Cross Entropy | binary_crossentropy | nn.BCELoss |
| 多クラス分類 | ソフトマックス交差エントロピー | Softmax Cross Entropy | categorical_crossentropy (one-hot vector用)<br> sparse_categorical_crossentropy | nn.CrossEntropyLoss |

In [ ]:
# 損失関数には、ソフトマックス交差エントロピー誤差を使用
# 今回は正解ラベルは one-hotベクトル として準備していることから、
# keras.losses.CategoricalCrossentropy() を使用する
# one-hotベクトルでない場合は、
# keras.losses.SparseCategoricalCrossentropy() を使用する
loss_fn = keras.losses.CategoricalCrossentropy()

In [ ]:
# 今の場合、予測値はいずれも 0.1 程度
# 交差エントロピー誤差は、-log(予測値) で表現される（前セクションを参照）
# 損失は、-log(0.1) ≒ 2.3 程度になるはず
loss_fn(train_labels, predictions).numpy()

In [ ]:
# 参考
# -log(0.1) を計算
-np.log(0.1)

In [ ]:
# model.compile で損失関数、オプティマイザを指定する
# 損失関数は先程定義した交差エントロピー誤差を使用する
# 最適化関数(オプティマイザ)として、Adam, RMSpropなどがある。ここでは、RMSprop を選択する
# モデルの評価は 後ほど、accuracy で行う

model.compile(loss = loss_fn,
              optimizer='rmsprop',
              metrics = ['accuracy'])

### 2-6. 学習・評価
- `model.fit()` で学習させる
- この時、訓練データと訓練データの正解ラベルをモデルに与える
- `validation_split` で訓練データのうち検証に使う割合を指定する
- `batch_size` でバッチサイズを指定する
- `epochs` で何回学習するかを指定する
- 学習の結果を変数 history に代入してあとで可視化する

In [ ]:
# loss: 損失
# accuracy: 正答率
# 10回の繰り返しで、loss が少しずつ減少、accuracy は増加

# 訓練データ, 訓練データの正解ラベル をまず入力 (train_images, train_labels)
# 訓練データの2割を検証データとして使用 (validation_split=0.2)
# ミニバッチ学習として、バッチサイズは128に設定 (batch_size=128)
#   128枚ずつ学習していくということ
# 繰り返し回数は10回 (epochs=10)

# 走らせると、損失値 loss が少しずつ小さくなり、正確度 accuracy が改善していくことに着目
history = model.fit(train_images,train_labels,
                    validation_split=0.2,
                    batch_size=128,
                    epochs=10)

- `model.evaluate(テストデータ,テストラベル)`を使うことで、modelの性能を表示できる

In [ ]:
# model.evaluateの引数に test_images, test_labels を指定
# verbose =1 とすると、学習のときと同じような結果表示になる
model.evaluate(test_images,test_labels, verbose=1)

## 3. 学習の視覚化
- matplotlib を用いて学習の様子を視覚化する
- 変数 history.history の中に loss と accuracy の10回の値が格納されている

In [ ]:
# history.historyの中を見てみる
# ディクショナリ型
# キーが 'loss', 'accuracy', 'val_loss', 'val_accuracy'
# val_ は 検証データでの結果
# 値が 損失値と正答率の10回の学習での推移
history.history

In [ ]:
# 訓練データの損失値 loss と検証データの損失値 val_loss をグラフとして表示
# 訓練データの loss の値を取り出して train_loss に代入
# ディクショナリ型の値は 変数名['キー名'] で取り出せる
train_loss = history.history['loss']
# 同様に検証データの loss の値を取り出して val_loss に代入
val_loss = history.history['val_loss']

# train_loss と val_loss をプロットする
plt.plot(train_loss, label='training')
plt.plot(val_loss, label='validation')
# グラフのタイトル
plt.title('loss over epochs')
# x軸の名前
plt.xlabel('epochs')
# y軸の名前
plt.ylabel('loss')
# 凡例
plt.legend()
# これらをすべてまとめて表示
plt.show()

In [ ]:
# 訓練データの正答率 accuracy と検証データの正答率 val_accuracy をグラフとして表示
# 訓練データの accuracy を取り出して train_accuracy に代入
train_accuracy = history.history['accuracy']
# 同様に検証データの accuracy を取り出して val_accuracy に代入
val_accuracy = history.history['val_accuracy']

# train_accuracy と val_accuracy をプロットする
plt.plot(train_accuracy, label='training')
plt.plot(val_accuracy, label='validation')
# グラフのタイトル
plt.title('accuracy over epochs')
# x軸の名前
plt.xlabel('epochs')
# y軸の名前
plt.ylabel('accuracy')
# 凡例
plt.legend()
# これらをすべてまとめて表示
plt.show()

## 4. ハイパーパラメータ
- 深層学習の実装の例を示したが、自身のデータで解析する際、人間が設定しなければならないパラメータがいくつかある
- これらを**ハイパーパラメータ**という
- 具体的には以下のようなものが挙げられる
    - 中間層のユニット数
    - Dropout率
    - 中間層の活性化関数
    - 損失関数
    - 最適化関数
    - バッチサイズ
    - エポック数
- より精度の高いモデルを構築するために、これらを吟味していくことが必要となる

### 演習問題

- 以下のパラメータでモデルを構築し、学習させた時、テストデータの正答率がどう変わるかを見てみてください

- 中間層のユニット数: 16
- 中間層の活性化関数: 'relu' ではなく 'sigmoid'

In [ ]:
# 必要なパッケージのインポート
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

# データの準備
from tensorflow.keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# データの正規化
train_images = train_images / 255.0
test_images = test_images / 255.0

# 変数 model を初期化
model = []

# モデルの構築
model = Sequential()
model.add(layers.Flatten(input_shape=(28, 28)))
model.add(layers.Dense(ここに代入, activation=ここに代入)) #中間層のユニット数(16)と活性化関数('sigmoid')
model.add(layers.Dropout(0.2))
model.add(layers.Dense(10))
model.add(layers.Softmax())

# モデルの要約
model.summary()




In [ ]:
# モデルの最適化

# 損失関数には、交差エントロピー誤差を使用
# 正解ラベルを one-hot ベクトルに変換していないため、
# SparseCategoricalCrossentropy()を使う
loss_fn = keras.losses.SparseCategoricalCrossentropy()


# 最適化関数には、RMSprop を使用
model.compile(loss = loss_fn,
              optimizer='rmsprop',
              metrics = ['accuracy'])


# モデルの学習
history = model.fit(train_images,train_labels,
                    validation_split=0.2,
                    batch_size=128,
                    epochs=10)

# loss の推移
train_loss = history.history['loss']
val_loss = history.history['val_loss']
plt.plot(train_loss, label='training')
plt.plot(val_loss, label='validation')
plt.title('loss over epochs')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

# accuracy の推移
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
plt.plot(train_accuracy, label='training')
plt.plot(val_accuracy, label='validation')
plt.title('accuracy over epochs')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()
plt.show()

# テストデータでの評価
model.evaluate(test_images,test_labels, verbose=1)


## 5. 畳み込みニューラルネットワーク Convolutional Neural Network (CNN)

- 全結合層のニューラルネットワークよりも精度が向上
- 畳み込みとプーリングを複数回施行していく
    - イメージ: 1回目の畳み込みで「斜め」「横」「縦」などの形状を学習、プーリングで特徴量マップをダウンサンプリング、2回めの畳み込みで、「斜めと横の組み合わせ」など、もう少し大きな形状を学習、プーリングで特徴量マップをさらにダウンサンプリング、3回目の畳み込みで、全体の空間的な位置関係を学習
- CNNの入力は(画像の縦のピクセル数, 画像の横のピクセル数, 画像のチャンネル数) で指定する
    - MNISTの手書き数字の画像は、縦28, 横28, 白黒(チャンネルの次元数は1) なので、(28,28,1)となる
- 畳み込みに使うフィルタの種類を filters で設定する。filters=32は、32種類のフィルタを使用するということである。kernel_size=3 は、フィルタの大きさが3x3の行列ということを示す
    - 32種類のフィルタを使うということは、特徴量マップは32種類できるということである。
- 最大プーリングに使う pool_size は特徴量マップを小さく分割する量を指定する。pool_size=2 は、特徴量マップを2x2ごとに分割し、各領域の最大値から新たな出力特徴量マップを生成する
- 以下を実行し、出力されるモデルの要約と上の説明を比較すると理解が進む

In [ ]:
# 必要なモジュールは全結合層の時と同じ
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

# モデルを初期化する
model = []

# Sequential関数を使って、モデルに層を設定していく
model = Sequential()

# 32種類の 3x3 のフィルタを使って畳み込みを行う
model.add(layers.Conv2D(filters=32, kernel_size=3, activation='relu',
                        input_shape=(28, 28, 1)))
# 2x2 で分割し最大値プーリング演算を行う
# これにより、出力される特徴量マップは 13x13x32 になる
model.add(layers.MaxPooling2D(pool_size=2))

# 64種類の 3x3 のフィルタを使って畳み込みを行う
model.add(layers.Conv2D(filters=64, kernel_size=3, activation='relu'))
# 2x2 で分割し最大値プーリング演算を行う
# これにより、出力される特徴量マップは 5x5x64 になる
model.add(layers.MaxPooling2D(pool_size=2))

# 128種類の 3x3 のフィルタを使って畳み込みを行う
# 特徴量マップは 3x3x128 となり、縦と横は十分小さいので、プーリングは必要ない
model.add(layers.Conv2D(filters=128, kernel_size=3, activation='relu'))

# 全結合層に投入するために、3次元のデータを1次元に変換する
model.add(layers.Flatten())

# ソフトマックス関数を使って、10クラス分類を行う
model.add(layers.Dense(10, activation='softmax'))

model.summary()



- CNNの計算は時間がかかる
- Google Colabでは、GPUを使用することができる
    - メニューの「ランタイム」→「ランタイムのタイプを変更」
    - ハードウェアアクセラレータを"GPU"に変更

<img src="https://www.nemotos.net/nb/img/colab_gpu.png" width="300">

In [ ]:
# データの準備
# mnistデータセットを入手
from tensorflow.keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# データの正規化
# 訓練データを、3次元のテンソルが60000枚スタックしているという形状に変更する
train_images = train_images.reshape((60000, 28, 28, 1))
# 画素値を 0-255 から、 0-1 に正規化する
train_images = train_images / 255.0

# テストデータも同様
test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images / 255.0

# モデルの最適化
# オプティマイザはRMSpropを使用する
# 損失関数は sparse_categorical_crossentropy を使用する
# モデルの評価には accuracy を使用する
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# モデルの学習
# 学習の際に訓練データの2割を検証データに使用する
# ミニバッチ学習を行う。バッチサイズは128
# エポック数は10とし、10回学習させる
history = model.fit(train_images,train_labels,
                    validation_split=0.2,
                    batch_size=128,
                    epochs=10)

# loss の推移
train_loss = history.history['loss']
val_loss = history.history['val_loss']
plt.plot(train_loss, label='training')
plt.plot(val_loss, label='validation')
plt.title('loss over epochs')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend()
plt.show()

# accuracy の推移
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
plt.plot(train_accuracy, label='training')
plt.plot(val_accuracy, label='validation')
plt.title('accuracy over epochs')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()
plt.show()

In [ ]:
# テストデータでの評価
# 全結合層のときよりも、結果が改善していることに着目
model.evaluate(test_images,test_labels, verbose=1)

## 復習

- 授業では手書き文字の識別を行いました
- fashion-mnist といって、洋服や小物などを集めた画像のデータベースがあります

| <img src="https://www.nemotos.net/nb/img/fashion-mnist-sprite.png" width=400> |
| --: |
| [https://github.com/zalandoresearch/fashion-mnist](https://github.com/zalandoresearch/fashion-mnist)より引用 |

- MNISTの手書き文字と同じサイズで、訓練データとテストデータの数も同じです
- 種類も10クラスです
    0. T-シャツ/トップ (T-shirt/top)
    1. ズボン (Trouser)
    2. プルオーバー (Pullover)
    3. ドレス (Dress)
    4. コート (Coat)
    5. サンダル (Sandal)
    6. シャツ (Shirt)
    7. スニーカー (Sneaker)
    8. バッグ (Bag)
    9. アンクルブーツ (Ankle boot)
- 全結合層とCNNでどれだけ結果が違うか検証してみましょう



In [ ]:
# 全結合層

##### 必要なモジュールのインポート
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential


##### データの準備
# fashion-mnistデータセットを入手
from tensorflow.keras.datasets import fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# データの正規化
# 訓練データを、3次元のテンソルが60000枚スタックしているという形状に変更する
train_images = train_images.reshape((60000, 28, 28, 1))
# 画素値を 0-255 から、 0-1 に正規化する
train_images = train_images / 255.0

# テストデータも同様
test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images / 255.0

##### 全結合層によるモデルの構築 (forward propagationの設定)
# (全結合層とCNNの違いはここだけ！)
# モデルを初期化する
model = []

# Sequential関数を使って、モデルに層を設定していく
model = Sequential()

# 28x28の画像を1次元に変換する
model.add(layers.Flatten(input_shape=(28, 28)))

# 隠れ層のユニット数は128ユニット、活性化関数はReLU
model.add(layers.Dense(128, activation='relu'))

# Dropout率を0.2で設定
model.add(layers.Dropout(0.2))

# 出力層は10クラス分類なので10、ソフトマックス関数で分類
model.add(layers.Dense(10))
model.add(layers.Softmax())

# モデルのサマリを表示
print('Model Summary')
print(model.summary())



In [ ]:
#### モデルの最適化 (backward propagationの設定)
# オプティマイザはRMSpropを使用する
# 損失関数は sparse_categorical_crossentropy を使用する
# モデルの評価には accuracy を使用する
model.compile(optimizer='rmsprop',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


#### モデルの学習
# history の初期化
history = []

# 学習の際に訓練データの2割を検証データに使用する
# ミニバッチ学習を行う。バッチサイズは128
# エポック数は10とし、10回学習させる
history = model.fit(train_images,train_labels,
                    validation_split=0.2,
                    batch_size=128,
                    epochs=10)


##### 損失値のグラフ表示
# 訓練データの損失値 loss と検証データの損失値 val_loss をグラフとして表示
# 訓練データの loss の値を取り出して train_loss に代入
# ディクショナリ型の値は 変数名['キー名']　で取り出せる
train_loss = history.history['loss']
# 同様に検証データの loss の値を取り出して val_loss に代入
val_loss = history.history['val_loss']

# train_loss と val_loss をプロットする
plt.plot(train_loss, label='training')
plt.plot(val_loss, label='validation')
# グラフのタイトル
plt.title('loss over epochs')
# x軸の名前
plt.xlabel('epochs')
# y軸の名前
plt.ylabel('loss')
# 凡例
plt.legend()
# これらをすべてまとめて表示
plt.show()


##### 正答率 のグラフ表示
# 訓練データの accuracy を取り出して train_accuracy に代入
train_accuracy = history.history['accuracy']
# 同様に検証データの accuracy を取り出して val_accuracy に代入
val_accuracy = history.history['val_accuracy']

# train_accuracy と val_accuracy をプロットする
plt.plot(train_accuracy, label='training')
plt.plot(val_accuracy, label='validation')
# グラフのタイトル
plt.title('accuracy over epochs')
# x軸の名前
plt.xlabel('epochs')
# y軸の名前
plt.ylabel('accuracy')
# 凡例
plt.legend()
# これらをすべてまとめて表示
plt.show()

#### テストデータでの評価
print('Evaluate with test data')
model.evaluate(test_images,test_labels, verbose=1)


In [ ]:
# CNN

##### 必要なモジュールのインポート
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential


##### データの準備
# fashion-mnistデータセットを入手
from tensorflow.keras.datasets import fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# データの正規化
# 訓練データを、3次元のテンソルが60000枚スタックしているという形状に変更する
train_images = train_images.reshape((60000, 28, 28, 1))
# 画素値を 0-255 から、 0-1 に正規化する
train_images = train_images / 255.0

# テストデータも同様
test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images / 255.0


##### CNNによるモデルの構築 (forward propagationの設定)
# 全結合層とCNNの違いはここだけ！
# モデルを初期化する
model = []

# Sequential関数を使って、モデルに層を設定していく
model = Sequential()

# 32種類の 3x3 のフィルタを使って畳み込みを行う
model.add(layers.Conv2D(filters=32, kernel_size=3, activation='relu',
                        input_shape=(28, 28, 1)))
# 2x2 で分割し最大値プーリング演算を行う
# これにより、出力される特徴量マップは 13x13x32 になる
model.add(layers.MaxPooling2D(pool_size=2))

# 64種類の 3x3 のフィルタを使って畳み込みを行う
model.add(layers.Conv2D(filters=64, kernel_size=3, activation='relu'))
# 2x2 で分割し最大値プーリング演算を行う
# これにより、出力される特徴量マップは 5x5x64 になる
model.add(layers.MaxPooling2D(pool_size=2))

# 128種類の 3x3 のフィルタを使って畳み込みを行う
# 特徴量マップは 3x3x128 となり、縦と横は十分小さいので、プーリングは必要ない
model.add(layers.Conv2D(filters=128, kernel_size=3, activation='relu'))

# 全結合層に投入するために、3次元のデータを1次元に変換する
model.add(layers.Flatten())

# ユニット数128の隠れ層を使用する。活性化関数は ReLU
model.add(layers.Dense(128,activation='relu'))

# Dropout率を0.2で設定
model.add(layers.Dropout(0.2))

# ソフトマックス関数を使って、10クラス分類を行う
model.add(layers.Dense(10, activation='softmax'))

# モデルのサマリを表示
print('Model Summary')
print(model.summary())




In [ ]:
#### モデルの最適化 (backward propagationの設定)
# オプティマイザはRMSpropを使用する
# 損失関数は sparse_categorical_crossentropy を使用する
# モデルの評価には accuracy を使用する
model.compile(optimizer='rmsprop',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


#### モデルの学習
# history.history の初期化
# hisotry.historyは dict型
history=[]

# 学習の際に訓練データの2割を検証データに使用する
# ミニバッチ学習を行う。バッチサイズは128
# エポック数は10とし、10回学習させる
history = model.fit(train_images,train_labels,
                    validation_split=0.2,
                    batch_size=128,
                    epochs=10)


##### 損失値のグラフ表示
# 訓練データの損失値 loss と検証データの損失値 val_loss をグラフとして表示
# 訓練データの loss の値を取り出して train_loss に代入
# ディクショナリ型の値は 変数名['キー名']　で取り出せる
train_loss = history.history['loss']
# 同様に検証データの loss の値を取り出して val_loss に代入
val_loss = history.history['val_loss']

# train_loss と val_loss をプロットする
plt.plot(train_loss, label='training')
plt.plot(val_loss, label='validation')
# グラフのタイトル
plt.title('loss over epochs')
# x軸の名前
plt.xlabel('epochs')
# y軸の名前
plt.ylabel('loss')
# 凡例
plt.legend()
# これらをすべてまとめて表示
plt.show()


##### 正答率 のグラフ表示
# 訓練データの accuracy を取り出して train_accuracy に代入
train_accuracy = history.history['accuracy']
# 同様に検証データの accuracy を取り出して val_accuracy に代入
val_accuracy = history.history['val_accuracy']

# train_accuracy と val_accuracy をプロットする
plt.plot(train_accuracy, label='training')
plt.plot(val_accuracy, label='validation')
# グラフのタイトル
plt.title('accuracy over epochs')
# x軸の名前
plt.xlabel('epochs')
# y軸の名前
plt.ylabel('accuracy')
# 凡例
plt.legend()
# これらをすべてまとめて表示
plt.show()

#### テストデータでの評価
print('Evaluate with test data')
model.evaluate(test_images,test_labels, verbose=1)
